<a href="https://colab.research.google.com/github/Jacquesjh/Absenteeism/blob/main/Absenteesim_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sempre, e antes de qualquer coisa, é importante aqui no Colab de nos certificarmos onde estamos trabalhando, e onde nossos arquivos gerados serão salvos
##Para esta análise, eu criei uma parta específica em meu Google Drive

In [1]:
%cd /content/drive/MyDrive/Machine Learning/Absenteeism

/content/drive/MyDrive/Machine Learning/Absenteeism


##Agora podemos começar a nossa análise. Para isso, começamos importando as bibliotecas relevantes. Inicialmente, importamos Pandas, pois queremos trabalhar com um Data Frame. Poderiamos utilizar Numpy, ao invés de Pandas, mas então estariamos trabalhando com dados do tipo Numpy Array.

In [2]:
import pandas as pd

##Agora podemos fazer o upload de nossos dados. O arquivo que queremos já está em nossa pasta que estamos trabalhamos e se chama 'Absenteeism_data' salvo como .csv

In [84]:
raw_data = pd.read_csv('Absenteeism_data.csv')

###Utilizarmos a biblioteca Pandas para tratar nossos dados como um DataFrame é muito benéfico para nos, pois preserva a estrutura dos dados, juntamente com o cabeçalho que nos dá uma boa ideia da natureza dos dados.
###Também, deste modo, podemos utilizar alguns métodos que ajudam no tratamento e visualização de dados. Isto também seria possível usando Numpy, porém seria de uma maneira diferente, pois teríamos que desconsiderar a primeira linha (contendo o cabeçalho) e utilizar outras função com os dados

In [85]:
##Usando esta função, podemos ver uma amostra de como estão organizados nossos dados
raw_data.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [86]:
##Com esta função, podemos ver algumas características estatísticas de cada feature 
raw_data.describe()

,ID,Reason for Absence,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,17.951429,19.411429,222.347143,29.892857,36.417143,271.801774,26.737143,1.282857,1.021429,0.687143,6.761429
std,11.028144,8.356292,66.312960,14.804446,6.379083,40.021804,4.254701,0.668090,1.112215,1.166095,12.670082
min,1.000000,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,1.000000,0.000000,0.000000,0.000000
25%,9.000000,13.000000,179.000000,16.000000,31.000000,241.476000,24.000000,1.000000,0.000000,0.000000,2.000000
50%,18.000000,23.000000,225.000000,26.000000,37.000000,264.249000,25.000000,1.000000,1.000000,0.000000,3.000000
75%,28.000000,27.000000,260.000000,50.000000,40.000000,294.217000,31.000000,1.000000,2.000000,1.000000,8.000000
max,36.000000,28.000000,388.000000,52.000000,58.000000,378.884000,38.000000,4.000000,4.000000,8.000000,120.000000


In [87]:
##Com esta função, podemos verificar os tipos de dados de cada feature, e também
##se temos algum valor nulo em nossos dados
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


##É importante criarmos uma versão dos nossos dados que não será alterada no pré processamento, assim, mais adiante, poderemos comparar a versão pré processada com a versão original.

In [88]:
original_data = raw_data.copy()

In [89]:
original_data

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
695,17,10,23/05/2018,179,22,40,237.656,22,2,2,0,8
696,28,6,23/05/2018,225,26,28,237.656,24,1,1,2,3
697,18,10,24/05/2018,330,16,28,237.656,25,2,0,0,8
698,25,23,24/05/2018,235,16,32,237.656,25,3,0,0,2


#Agora que temos os nossos dados em uma estrutura trabalhável, podemos nos ater à tarefa em sí: Preparar nossos dados para um regressão logística de abstenção, isto é, criar um modelo que consiga prever, baseado em outras características, se um indivíduo estará ausento do trabalho no futuro.
##Deste modo, fica claro que nossa variável dependente, ou "target", ou variável controlada é "Absenteesim Time in Hours".
##E nossos outros dados serão os "inputs" para esta regressão logística.

#Com isto em mente, podemos começar analisando a natureza dos nossos features/inputs
##ID: é apenas a identificação do indivíduo, além, disso, podemos perceber que ela não é exclusiva de apenas um indivíduo. Como a natureza dela não nos diz nada relevante, podemos excluí-la de nossos dados

In [90]:
raw_data = raw_data.drop(['ID'], axis = 1)

In [91]:
raw_data.head()

,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


##Reason for Absence: este feature nos diz sobre as diferentes razões pela quais alguem estava ausente. Os valores dela são tabeladas, ou seja, o valor 12 para este feature traduz como "Motivo de Saúde", por exemplo, ou então "Motivo de Pessoal". Assim, ela é uma variável CATEGORICAL NOMINAL, sem sentido numérico.

In [92]:
##Podemos investigar os motivos mais frequentes de abstenção
raw_data['Reason for Absence'].value_counts()

23    147
28    110
27     66
13     52
0      38
     ... 
4       2
15      2
3       1
2       1
17      1
Name: Reason for Absence, Length: 28, dtype: int64

In [93]:
##Como este tipo de dado é categórico, uma análise estatística dos features
##não faz sentido, pois eles não tem relação numérica.
##Assim, usar uma função como a "describe" não fornece informação alguma
raw_data['Reason for Absence'].describe()

count    700.000000
mean      19.411429
std        8.356292
min        0.000000
25%       13.000000
50%       23.000000
75%       27.000000
max       28.000000
Name: Reason for Absence, dtype: float64

###Como nosos objetivo é investigar se algum indivíduo se encontrou ausente, e não por qual motivo ele se encontrou ausente, podemos transformar este feature em 0 = estava ausente por motivo desconhecido e 1 = estava ausente por algum motivo

In [94]:
reasons = pd.get_dummies(raw_data['Reason for Absence'])

In [95]:
reasons

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [96]:
reasons = reasons.drop(0, axis = 1)

In [97]:
reasons

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


###Até aqui nos criamos um data frame com os todos os motivos conhecidos de ausência. Agora nos iremos excluir o feature "Reasons for Absence" e substituir por grupos que contenham esses data frame
###Podemos agrupar os motivos em algumas categorias.

In [98]:
raw_data = raw_data.drop(['Reason for Absence'], axis = 1)

In [99]:
raw_data

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,23/07/2015,289,36,33,239.554,30,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...
695,23/05/2018,179,22,40,237.656,22,2,2,0,8
696,23/05/2018,225,26,28,237.656,24,1,1,2,3
697,24/05/2018,330,16,28,237.656,25,2,0,0,8
698,24/05/2018,235,16,32,237.656,25,3,0,0,2


In [102]:
reasons_type1 = reasons.loc[:, 1:14].max(axis = 1)
reasons_type2 = reasons.loc[:, 15:17].max(axis = 1)
reasons_type3 = reasons.loc[:, 18:21].max(axis = 1)
reasons_type4 = reasons.loc[:, 22:].max(axis = 1)

In [105]:
reasons_type3

0      0
1      0
2      0
3      0
4      0
      ..
695    0
696    0
697    0
698    0
699    0
Length: 700, dtype: uint8

In [106]:
##Agora nos juntamos estes 4 tipos de razões com o resto de nossos dados
new_data = pd.concat([raw_data, reasons_type1, reasons_type2, reasons_type3, reasons_type4], axis = 1)
new_data

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,0,1,2,3
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,23/05/2018,179,22,40,237.656,22,2,2,0,8,1,0,0,0
696,23/05/2018,225,26,28,237.656,24,1,1,2,3,1,0,0,0
697,24/05/2018,330,16,28,237.656,25,2,0,0,8,1,0,0,0
698,24/05/2018,235,16,32,237.656,25,3,0,0,2,0,0,0,1


In [108]:
##Renomeamos os features criados
new_data.columns.values

array(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 0, 1, 2, 3],
      dtype=object)

In [109]:
column_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 'Reason 1', 'Reason 2', 'Reason 3', 'Reason 4']

In [111]:
new_data.columns = column_names

In [112]:
new_data.head()

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Reason 1,Reason 2,Reason 3,Reason 4
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [113]:
##Agora podemos reorganizar as colunas dos nossos dados
column_names_reordered = ['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

In [116]:
new_data = new_data[column_names_reordered]
new_data.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [166]:
data_reason_mod = new_data.copy()

,Reason 1,Reason 2,Reason 3,Reason 4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [167]:
data_reason_mod.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


##Date: A data em que houve alguma ausência. Este dado é bastante importante para nós, pois podemos extrair alguns padrões de ausência, como dias perto de feriados, após feriados, evento importantes, etc. 

In [168]:
##O arquivo que estamos trabalhando até agora é "data_reason_mod"
data_reason_mod.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [169]:
data_reason_mod['Date'].describe()

count            700
unique           432
top       24/10/2017
freq               5
Name: Date, dtype: object

In [170]:
##Podemos verificar que tipo de estrutura cada dado do feature
type(data_reason_mod['Date'][0])

str

###Como os dados estão do tipo "dd/mm/aaaa" numa estrutura de string, podemos converter estes dados para algo mais facilmente trabalhavel

In [171]:
data_reason_mod['Date'] = pd.to_datetime(data_reason_mod['Date'], format = '%d/%m/%Y')
data_reason_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Reason 1                   700 non-null    uint8         
 1   Reason 2                   700 non-null    uint8         
 2   Reason 3                   700 non-null    uint8         
 3   Reason 4                   700 non-null    uint8         
 4   Date                       700 non-null    datetime64[ns]
 5   Transportation Expense     700 non-null    int64         
 6   Distance to Work           700 non-null    int64         
 7   Age                        700 non-null    int64         
 8   Daily Work Load Average    700 non-null    float64       
 9   Body Mass Index            700 non-null    int64         
 10  Education                  700 non-null    int64         
 11  Children                   700 non-null    int64         
 12  Pets    

In [172]:
##Podemos extrair o mes, dia e outras informações através dos metodos .month e .day
print(data_reason_mod['Date'][0].day)
print(data_reason_mod['Date'][0].month)
print(data_reason_mod['Date'][0].weekday())
print(data_reason_mod['Date'][0].year)

7
7
1
2015


###Podemos criar novos features nos nossos dados com estas informações separadas

In [173]:
month_list = []
weekday_list = []

for i in range(data_reason_mod['Date'].shape[0]):
  weekday_list.append(data_reason_mod['Date'][i].weekday())
  month_list.append(data_reason_mod['Date'][i].month)

In [174]:
data_reason_mod['Day of the Week'] = weekday_list
data_reason_mod['Month Value'] = month_list

In [175]:
data_reason_mod.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Day of the Week,Month Value
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,1,7
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,1,7
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,2,7
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,3,7
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,3,7


In [176]:
data_reason_mod = data_reason_mod.drop(['Date'], axis = 1)

In [177]:
new_columns_name = ['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Day of the Week', 'Month Value', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

In [178]:
data_reason_mod = data_reason_mod[new_columns_name]

In [179]:
data_reason_mod

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,1,7,118,13,50,239.554,31,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,0,0,0,1,3,5,235,16,32,237.656,25,3,0,0,2
699,0,0,0,1,3,5,291,31,40,237.656,25,1,1,1,2


In [180]:
data_time_mod = data_reason_mod

##Transportation Expense: é o custo médio mensal da jornada de trabalho da pessoa. Podemos normaliza-lá para um intervalor de [0,1].

In [196]:
data_time_mod.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,1,7,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,2,7,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,3,7,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,3,7,289,36,33,239.554,30,1,2,1,2


In [197]:
type(data_time_mod['Transportation Expense'][0])

numpy.int64

In [199]:
max_value = data_time_mod['Transportation Expense'].max()

In [200]:
def normalize(trans_value):
  return trans_value/max_value

In [201]:
data_time_mod['Transportation Expense'] = data_time_mod['Transportation Expense'].apply(normalize)

In [202]:
data_time_mod.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,0.744845,36,33,239.554,30,1,2,1,4
1,0,0,0,0,1,7,0.304124,13,50,239.554,31,1,1,0,0
2,0,0,0,1,2,7,0.461340,51,38,239.554,31,1,0,0,2
3,1,0,0,0,3,7,0.719072,5,39,239.554,24,1,2,0,4
4,0,0,0,1,3,7,0.744845,36,33,239.554,30,1,2,1,2


In [203]:
data_trans_norm = data_time_mod

##Distance to Work: é a distáncia que o indivíduo percorre até chegar ao trabalho. Será normalizado da mesma maneira que Transportation Expense

In [204]:
data_trans_norm.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,0.744845,36,33,239.554,30,1,2,1,4
1,0,0,0,0,1,7,0.304124,13,50,239.554,31,1,1,0,0
2,0,0,0,1,2,7,0.461340,51,38,239.554,31,1,0,0,2
3,1,0,0,0,3,7,0.719072,5,39,239.554,24,1,2,0,4
4,0,0,0,1,3,7,0.744845,36,33,239.554,30,1,2,1,2


In [205]:
max_value = data_trans_norm['Distance to Work'].max()

def normalize(dis_value):
  return dis_value/max_value

In [206]:
data_trans_norm['Distance to Work'] = data_trans_norm['Distance to Work'].apply(normalize)

In [207]:
data_trans_norm.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,0.744845,0.692308,33,239.554,30,1,2,1,4
1,0,0,0,0,1,7,0.304124,0.250000,50,239.554,31,1,1,0,0
2,0,0,0,1,2,7,0.461340,0.980769,38,239.554,31,1,0,0,2
3,1,0,0,0,3,7,0.719072,0.096154,39,239.554,24,1,2,0,4
4,0,0,0,1,3,7,0.744845,0.692308,33,239.554,30,1,2,1,2


In [208]:
data_dist_norm = data_trans_norm

##Age: é a idade de cada indivíduo. Podemos normalizar este feature também

In [209]:
max_value = data_dist_norm['Age'].max()

def normalize(age):
  return age/max_value

In [210]:
data_dist_norm['Age'] = data_dist_norm['Age'].apply(normalize)

In [212]:
data_dist_norm.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,0.744845,0.692308,0.568966,239.554,30,1,2,1,4
1,0,0,0,0,1,7,0.304124,0.250000,0.862069,239.554,31,1,1,0,0
2,0,0,0,1,2,7,0.461340,0.980769,0.655172,239.554,31,1,0,0,2
3,1,0,0,0,3,7,0.719072,0.096154,0.672414,239.554,24,1,2,0,4
4,0,0,0,1,3,7,0.744845,0.692308,0.568966,239.554,30,1,2,1,2


In [213]:
data_age_norm = data_dist_norm

##Daily Work Load Average: é a quantidade de minutos trabalhados por dia em média. Iresmo normalizar este feature também

In [214]:
max_value = data_age_norm['Daily Work Load Average'].max()

def normalize(age):
  return age/max_value

In [215]:
data_age_norm['Daily Work Load Average'] = data_age_norm['Daily Work Load Average'].apply(normalize)

In [216]:
data_age_norm.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,0.744845,0.692308,0.568966,0.632262,30,1,2,1,4
1,0,0,0,0,1,7,0.304124,0.250000,0.862069,0.632262,31,1,1,0,0
2,0,0,0,1,2,7,0.461340,0.980769,0.655172,0.632262,31,1,0,0,2
3,1,0,0,0,3,7,0.719072,0.096154,0.672414,0.632262,24,1,2,0,4
4,0,0,0,1,3,7,0.744845,0.692308,0.568966,0.632262,30,1,2,1,2


In [217]:
data_work_norm = data_age_norm

##Body Mass Index: é o índice de massa corporal do indivíduo. Iremos normalizer também.

In [218]:
max_value = data_work_norm['Body Mass Index'].max()

def normalize(body):
  return body/max_value

In [219]:
data_work_norm['Body Mass Index'] = data_work_norm['Body Mass Index'].apply(normalize)

In [220]:
data_work_norm.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,0.744845,0.692308,0.568966,0.632262,0.789474,1,2,1,4
1,0,0,0,0,1,7,0.304124,0.250000,0.862069,0.632262,0.815789,1,1,0,0
2,0,0,0,1,2,7,0.461340,0.980769,0.655172,0.632262,0.815789,1,0,0,2
3,1,0,0,0,3,7,0.719072,0.096154,0.672414,0.632262,0.631579,1,2,0,4
4,0,0,0,1,3,7,0.744845,0.692308,0.568966,0.632262,0.789474,1,2,1,2


In [221]:
data_normalized = data_work_norm

##Education: é uma variável categoríca que corresponde ao nível de educação da pessoa, sendo: 1 - Ensino Médio, 2 - Ensino Superio, 3 - Pós Graduação e 4 - Doutorado/Mestrado

In [225]:
##Podemos verificar quantas pessoas se encaixam em cada categoria
data_normalized['Education'].value_counts()

1    583
3     73
2     40
4      4
Name: Education, dtype: int64

###Como a categoria predominante é de Ensino Médio, podemos juntar as outras 3 categorias em uma apenas. 0 - Ensino Médio e 1 - Ensino Superior ou Mais

In [226]:
##Para isto, utilizaremos um dicionário para implementar estas mudanças de índices
data_normalized['Education'] = data_normalized['Education'].map({1:0, 2:1, 3:1, 4:1})

In [227]:
data_normalized['Education'].value_counts()

0    583
1    117
Name: Education, dtype: int64

In [229]:
data_education_mod = data_normalized.copy()

##Children: é o número de filhos que o individuo possuí. Nenhum tipo de manipulação de dado será aplicado aqui.

##Pets: é o número de animais de estimação que o indivíduo possuí. Aqui também não é necessário nenhum tratamento de dado

#Com isso, terminamos o nosso pré processamento de dados, e estamos prontos para salvar nossos dados

In [230]:
data_preprocessed = data_education_mod.copy()

In [231]:
data_preprocessed.head(10)

,Reason 1,Reason 2,Reason 3,Reason 4,Day of the Week,Month Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,1,7,0.744845,0.692308,0.568966,0.632262,0.789474,0,2,1,4
1,0,0,0,0,1,7,0.304124,0.250000,0.862069,0.632262,0.815789,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,0,0,1,0,0,7,0.399485,0.230769,0.586207,0.632262,0.657895,0,2,0,40
9,0,0,0,1,0,7,0.605670,0.211538,0.637931,0.632262,0.763158,1,1,1,8


In [232]:
##Iremos salvar nossos dados em nossa pasta de trabalho
data_preprocessed.to_csv('Absenteeism_preprocessed.csv', index = False)